In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 24.5 MB/s eta 0:00:00


In [3]:
from braindecode.datautil import create_from_X_y
import scipy.io
import numpy as np
from skorch.helper import predefined_split
from sklearn.model_selection import train_test_split
from braindecode.models import Deep4Net
from braindecode.datautil import create_from_X_y
from braindecode import EEGClassifier
import torch
import os
import scipy.io
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier
data_dir = '/content/drive/MyDrive/2dGrazNormalized/'


model = Deep4Net(
    in_chans=22,  # Number of input channels
    n_classes=4,  # Number of classes
    input_window_samples=1000,
    final_conv_length='auto'
)

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 32
n_epochs = 100
device = 'cuda'

for i in range(1,10):
    file_name1 = None
    file_name2 = None
    if i <= 9:
        file_name1 = f'norm_2d_{i:02d}T_P.mat'  # Construct the file name
        file_name2 = f'norm_2d_{i:02d}E_P.mat'  # Construct the file name
    else:
        file_name1 = f'trans_A{i:03d}T_P.mat'
        file_name2 = f'trans_A{i:03d}E_P.mat'

    file_path1 = os.path.join(data_dir, file_name1)  # Construct the file path
    file_path2 = os.path.join(data_dir, file_name2)  # Construct the file path
    training = scipy.io.loadmat(file_path1)
    testing = scipy.io.loadmat(file_path2)

    # Assuming you have your training and testing datasets
    train_X, train_y = training['data'], (np.squeeze(training['label'])).astype(int)-1
    valid_X, valid_y = testing['data'], (np.squeeze(testing['label'])).astype(int)-1

    # Create the train-validation split using train_test_split
    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

    # Create the dataset objects
    train_dataset = create_from_X_y(train_X, train_y, sfreq=250, drop_last_window=True)
    valid_dataset = create_from_X_y(valid_X, valid_y, sfreq=250, drop_last_window=True)

    # Create the predefined split using the validation dataset
    valid_split = predefined_split(valid_dataset)


    clf = EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_dataset),
        optimizer__lr=lr,
        optimizer__weight_decay=weight_decay,
        batch_size=batch_size,
        callbacks=[
            "accuracy",
            ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
        ],
        device=None,
    )

    clf.fit(train_dataset, y=None, epochs=n_epochs)

    with open('subject'+str(i)+'_history.txt', 'w') as file:
        for j in clf.history_:
            file.write('epoch:' + str(j['epoch']) + ' train_loss:' + str(j['train_loss']) + ' train_acc:' + str(j['train_accuracy']) + ' valid_loss:' + str(j['valid_loss']) + ' valid_acc:' + str(j['valid_accuracy']) + '\n')


/usr/local/lib/python3.10/dist-packages/braindecode/datautil/__init__.py:15: UserWarning: create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y
  warn('create_from_X_y has been moved to datasets, please use from braindecode.datasets import create_from_X_y')


Streaming output truncated to the last 5000 lines.
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded Raw for 1 events and 1000 original time points ...
Using data from preloaded

In [4]:
import shutil

# Specify the source directory where the text files are located
source_directory = '/content/'

# Specify the destination directory in your Google Drive
destination_directory = '/content/drive/MyDrive/ResultCCNGraz2d_3'

# Move all text files from the source directory to the destination directory
for file_name in os.listdir(source_directory):
    if file_name.endswith('.txt'):
        file_path = os.path.join(source_directory, file_name)
        shutil.move(file_path, destination_directory)

print("Text files moved to Google Drive successfully.")

Text files moved to Google Drive successfully.
